# Impact of mean-field assumption

In [1]:
import kickscore as ks
import numpy as np

from datetime import datetime, timezone, timedelta
from kseval.models import iterate_dataset
from kseval.models.base import YEAR
from kseval.models.basketball import DATASET
from kseval.regression import DynamicLinearRegression
from math import log

In [2]:
teams = [
    "ATL", "BOS", "CHH", "CHI", "CHO", "CLE", "DAL", "DEN", "DET", "GSW", "HOU",
    "IND", "LAC", "LAL", "MEM", "MIA", "MIL", "MIN", "NJN", "NOP", "NYK", "ORL",
    "PHI", "PHO", "POR", "SAC", "SAS", "SEA", "TOR", "UTA", "VAN", "WAS", 
]
M = len(teams)
team2id = dict(zip(teams, range(M)))

In [3]:
kern = (ks.kernel.Constant(var=0.05931)
        + ks.kernel.Exponential(var=17.66673, lscale=(3.31032*YEAR)))

begin = datetime(2000, 8, 1, tzinfo=timezone.utc).timestamp()
cutoff = datetime(2004, 8, 1, tzinfo=timezone.utc)
bounds = [(cutoff + timedelta(days=5*i)).timestamp() for i in range(70)]

## Full covariance

In [4]:
def evaluate(X1, t1, y1, X2, t2, y2):
    if len(X2) == 0:
        return 0, 0, 0
    n_obs, log_loss, accuracy = 0, 0, 0
    reg = DynamicLinearRegression(time_kern=kern, noise_var=143.45175)
    reg.fit(X1, t1, y1)
    prob = reg.probabilities(X2, t2)
    for y, p in zip(y2, prob):
        if y > 0:
            log_loss += -log(p)
            accuracy += 1.0 if p > 0.5 else 0.0
        else:
            log_loss += -log(1 - p)
            accuracy += 1.0 if 1 - p > 0.5 else 0.0
        n_obs += 1
    return n_obs, log_loss, accuracy

In [5]:
%%time
fc = {"n_obs": 0, "accuracy": 0, "log_loss": 0}

for t1, t2 in zip(bounds[:-1], bounds[1:]):
    print(".", end="")
    X_train, X_test = list(), list()
    t_train, t_test = list(), list()
    y_train, y_test = list(), list()
    for obs in iterate_dataset(DATASET):
        if obs["t"] < begin or obs["t"] >= t2:
            continue
        vec = np.zeros(M)
        vec[team2id[obs["team1"]]] = +1
        vec[team2id[obs["team2"]]] = -1
        if obs["t"] < t1:
            X_train.append(vec)
            y_train.append(obs["score1"] - obs["score2"])
            t_train.append(obs["t"])
        else:
            X_test.append(vec)
            y_test.append(obs["score1"] - obs["score2"])
            t_test.append(obs["t"])
    res = evaluate(
        np.array(X_train), np.array(t_train), np.array(y_train),
        np.array(X_test), np.array(t_test), np.array(y_test))
    fc["n_obs"] += res[0]
    fc["log_loss"] += res[1]
    fc["accuracy"] += res[2]
print()

.....................................................................
CPU times: user 6min 45s, sys: 1min 16s, total: 8min 2s
Wall time: 5min 3s


In [6]:
print("log loss: {:.3f}".format(fc["log_loss"] / fc["n_obs"]))
print("accuracy: {:.3f}".format(fc["accuracy"] / fc["n_obs"]))

log loss: 0.634
accuracy: 0.664


## Mean-field approximation

In [7]:
%%time
mf = {"n_obs": 0, "accuracy": 0, "log_loss": 0}

model = ks.DifferenceModel(var=143.45175)
for team in teams:
    model.add_item(team, kernel=kern)
    
for obs in iterate_dataset(DATASET):
    if obs["t"] < begin:
        continue
    elif obs["t"] < bounds[0]:
        diff = obs["score1"] - obs["score2"]
        model.observe([obs["team1"]], [obs["team2"]], diff=diff, t=obs["t"])
    else:
        break

for i in range(len(bounds) - 1):
    print(".", end="")
    fitted = False
    for obs in iterate_dataset(DATASET):
        if i == 0 or obs["t"] < bounds[i-1]:
            continue
        elif obs["t"] < bounds[i]:
            diff = obs["score1"] - obs["score2"]
            model.observe([obs["team1"]], [obs["team2"]], diff=diff, t=obs["t"])
        elif obs["t"] < bounds[i+1]:
            if not fitted:
                model.fit()
            diff = obs["score1"] - obs["score2"]
            p1, p2 = model.probabilities([obs["team1"]], [obs["team2"]], t=obs["t"])
            if diff > 0:
                mf["log_loss"] += -log(p1)
                mf["accuracy"] += 1.0 if p1 > 0.5 else 0.0
            else:
                mf["log_loss"] += -log(p2)
                mf["accuracy"] += 1.0 if p2 > 0.5 else 0.0
            mf["n_obs"] += 1
print()

.....................................................................
CPU times: user 11min 19s, sys: 1min 42s, total: 13min 2s
Wall time: 6min 58s


In [8]:
print("log loss: {:.3f}".format(mf["log_loss"] / mf["n_obs"]))
print("accuracy: {:.3f}".format(mf["accuracy"] / mf["n_obs"]))

log loss: 0.634
accuracy: 0.664


## Sanity check against GPy

In [9]:
X = np.array([
    [1.0, -1.0],
    [-1.0, 1.0],
])
t = np.array([0.0, 1.0])
y = np.array([1.7, 2.3])

kern = ks.kernel.Constant(var=1.0)
reg = DynamicLinearRegression(time_kern=kern, noise_var=0.5)

reg.fit(X, t, y)

-10.263342174517508

In [10]:
mu, var = reg.predict(X, t)
mu, var

(array([-0.26666667,  0.26666667]), array([0.72222222, 0.72222222]))

In [11]:
import GPy
kern = GPy.kern.Linear(input_dim=2)
m = GPy.models.GPRegression(X=X, Y=y[:,None], kernel=kern, noise_var=0.5)
m.log_likelihood()

-10.263342025584178

In [12]:
m.predict(X)

(array([[-0.26666667],
        [ 0.26666667]]), array([[0.72222223],
        [0.72222223]]))

Big case:

    import GPy
    kern = (GPy.kern.Linear(input_dim=32, active_dims=list(range(1, 33)))
            * (GPy.kern.Bias(input_dim=1, active_dims=[0], variance=0.05931)
                    + GPy.kern.Exponential(input_dim=1, active_dims=[0],
                            variance=17.66673, lengthscale=3.31032)))
    m = GPy.models.GPRegression(X=np.hstack((t[:,None]/YEAR, X)), Y=y[:,None], kernel=kern, noise_var=143.45175)
    m.log_likelihood()